## A Step-by-Step Guide to Feature Engineering for Multivariate Time Series

- https://towardsdatascience.com/a-step-by-step-guide-to-feature-engineering-for-multivariate-time-series-162ccf232e2f
- ml기반 시계열예측 3번째 포스트

<div style="text-align: right"> <b>Author : Kwang Myung Yu</b></div>
<div style="text-align: right"> Initial upload: 2023.6.25</div>
<div style="text-align: right"> Last update: 2023.6.25</div>

In [1]:
import re
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline
# print(plt.stype.available)

# Options for pandas
pd.options.display.max_columns = 30

스마트 부표에서 수집한 다변량 시계열을 사례 연구로 사용하겠습니다[1]. 이 부표는 아일랜드 해안에 설치되어 있습니다. 이 부표는 해양 조건과 관련된 9가지 변수를 수집합니다. 여기에는 해수 온도, 파도 높이, 해수 유속 등이 포함됩니다. 위의 그림 1은 2022년 첫 달의 플롯을 보여줍니다.

In [2]:
# 앞에 사용된 것보다 개선된 것
def time_delay_embedding(series: pd.Series,
                         n_lags: int,
                         horizon: int,
                         return_Xy: bool = False):
    """
    Time delay embedding
    Time series for supervised learning

    :param series: time series as pd.Series
    :param n_lags: number of past values to used as explanatory variables
    :param horizon: how many values to forecast
    :param return_Xy: whether to return the lags split from future observations

    :return: pd.DataFrame with reconstructed time series
    """
    assert isinstance(series, pd.Series)

    if series.name is None:
        name = 'Series'
    else:
        name = series.name

    n_lags_iter = list(range(n_lags, -horizon, -1))

    df_list = [series.shift(i) for i in n_lags_iter]
    df = pd.concat(df_list, axis=1).dropna()
    df.columns = [f'{name}(t-{j - 1})'
                  if j > 0 else f'{name}(t+{np.abs(j) + 1})'
                  for j in n_lags_iter]

    df.columns = [re.sub('t-0', 't', x) for x in df.columns]

    if not return_Xy:
        return df

    is_future = df.columns.str.contains('\+')

    X = df.iloc[:, ~is_future]
    Y = df.iloc[:, is_future]
    if Y.shape[1] == 1:
        Y = Y.iloc[:, 0]

    return X, Y

In [3]:
buoy = pd.read_csv("data/smart_buoy.csv", skiprows = [1], parse_dates=["time"])

In [4]:
buoy.head()

,time,station_id,PeakPeriod,PeakDirection,UpcrossPeriod,SignificantWaveHeight,SeaTemperature,Hmax,THmax,MeanCurDirTo,MeanCurSpeed
0,2022-01-01 00:00:00+00:00,SmartBay Wave Buoy,4.17,169.05495,3.96,154.0,10.16,272.0,4.91,0.351648,0.216
1,2022-01-01 00:05:00+00:00,SmartBay Wave Buoy,NaN,NaN,NaN,NaN,10.16,NaN,NaN,NaN,NaN
2,2022-01-01 00:10:00+00:00,SmartBay Wave Buoy,NaN,NaN,NaN,NaN,10.11,NaN,NaN,1.934066,0.170
3,2022-01-01 00:15:00+00:00,SmartBay Wave Buoy,NaN,NaN,NaN,NaN,10.11,NaN,NaN,NaN,NaN
4,2022-01-01 00:20:00+00:00,SmartBay Wave Buoy,NaN,NaN,NaN,NaN,10.11,NaN,NaN,1.054945,0.138


In [5]:
buoy = buoy.drop("station_id", axis = 1)
buoy = buoy.set_index("time")
buoy = buoy.resample("1H").mean()
buoy.columns = [
    'PeakP', 'PeakD', 'Upcross',
    'SWH', 'SeaTemp', 'Hmax', 'THmax',
    'MCurDir', 'MCurSpd'
]

In [6]:
buoy.head()

,PeakP,PeakD,Upcross,SWH,SeaTemp,Hmax,THmax,MCurDir,MCurSpd
time,,,,,,,,,
2022-01-01 00:00:00+00:00,4.010,162.153850,3.890,146.0,10.130833,267.5,4.675,178.051281,0.137000
2022-01-01 01:00:00+00:00,4.130,161.362635,3.925,142.5,10.135000,206.0,4.080,179.882784,0.126000
2022-01-01 02:00:00+00:00,4.085,158.461540,3.855,143.0,10.118333,257.5,4.535,295.648351,0.178667
2022-01-01 03:00:00+00:00,5.985,191.428575,4.075,153.5,10.143333,260.0,8.170,339.413908,0.224333
2022-01-01 04:00:00+00:00,5.970,221.186815,4.305,171.0,10.143333,292.0,4.995,288.586082,0.211667


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor

In [8]:
target_var = "SWH"
col_names = buoy.columns.tolist()

In [9]:
buoy_ds = []
for col in buoy:
    col_df = time_delay_embedding(buoy[col], n_lags = 24, horizon=12)
    buoy_ds.append(col_df)

In [10]:
buoy_df = pd.concat(buoy_ds, axis=1).dropna()
print(buoy_df.shape)
display(buoy_df.head())

(4790, 324)


,PeakP(t-23),PeakP(t-22),PeakP(t-21),PeakP(t-20),PeakP(t-19),PeakP(t-18),PeakP(t-17),PeakP(t-16),PeakP(t-15),PeakP(t-14),PeakP(t-13),PeakP(t-12),PeakP(t-11),PeakP(t-10),PeakP(t-9),...,MCurSpd(t-2),MCurSpd(t-1),MCurSpd(t),MCurSpd(t+1),MCurSpd(t+2),MCurSpd(t+3),MCurSpd(t+4),MCurSpd(t+5),MCurSpd(t+6),MCurSpd(t+7),MCurSpd(t+8),MCurSpd(t+9),MCurSpd(t+10),MCurSpd(t+11),MCurSpd(t+12)
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-01-02 00:00:00+00:00,4.010,4.130,4.085,5.985,5.970,7.180,8.515,7.380,7.380,7.275,7.415,9.030,6.255,9.890,9.030,...,0.050667,0.077000,0.129833,0.170500,0.161833,0.221500,0.201833,0.210833,0.126500,0.095167,0.072667,0.086167,0.082833,0.146833,0.185000
2022-01-02 01:00:00+00:00,4.130,4.085,5.985,5.970,7.180,8.515,7.380,7.380,7.275,7.415,9.030,6.255,9.890,9.030,6.645,...,0.077000,0.129833,0.170500,0.161833,0.221500,0.201833,0.210833,0.126500,0.095167,0.072667,0.086167,0.082833,0.146833,0.185000,0.237833
2022-01-02 02:00:00+00:00,4.085,5.985,5.970,7.180,8.515,7.380,7.380,7.275,7.415,9.030,6.255,9.890,9.030,6.645,9.005,...,0.129833,0.170500,0.161833,0.221500,0.201833,0.210833,0.126500,0.095167,0.072667,0.086167,0.082833,0.146833,0.185000,0.237833,0.270167
2022-01-02 03:00:00+00:00,5.985,5.970,7.180,8.515,7.380,7.380,7.275,7.415,9.030,6.255,9.890,9.030,6.645,9.005,6.905,...,0.170500,0.161833,0.221500,0.201833,0.210833,0.126500,0.095167,0.072667,0.086167,0.082833,0.146833,0.185000,0.237833,0.270167,0.296000
2022-01-02 04:00:00+00:00,5.970,7.180,8.515,7.380,7.380,7.275,7.415,9.030,6.255,9.890,9.030,6.645,9.005,6.905,8.195,...,0.161833,0.221500,0.201833,0.210833,0.126500,0.095167,0.072667,0.086167,0.082833,0.146833,0.185000,0.237833,0.270167,0.296000,0.248167


In [11]:
predictor_variables = buoy_df.columns.str.contains('\(t\-')
target_variables = buoy_df.columns.str.contains(f'{target_var}\(t\+')
X = buoy_df.iloc[:, predictor_variables]
y = buoy_df.iloc[:, target_variables]

# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)
# fitting a lgbm model without feature engineering
model_wo_fe = MultiOutputRegressor(LGBMRegressor())
model_wo_fe.fit(X_train, y_train)

# getting forecasts for the test set
preds_wo_fe = model_wo_fe.predict(X_test)

# computing the MAPE error
mape(y_test, preds_wo_fe)

0.2643572748139305

- 여기서는 MultioutputRegressor를 사용하여 direct approach를 사용하였다.  

이제 피처를 추가하여 성능을 개선할 수 있는지 살펴본다.  두가지 방법이 있다.  
- Univariate feature extraction :  Computing rolling statistics of each variable. For example, a rolling average can be used to smooth out spurious observations
- Bivariate feature extraction : Computing rolling statistics of pairs of variables to summarise their interaction. For example, the rolling covariance between two variables.

#### Univariate Feature Extraction

In [12]:
col_names

['PeakP',
 'PeakD',
 'Upcross',
 'SWH',
 'SeaTemp',
 'Hmax',
 'THmax',
 'MCurDir',
 'MCurSpd']

In [13]:
SUMMARY_STATS = {
    'mean': np.mean,
    'sdev': np.std,
}

univariate_features = {}
for col in col_names:
    X_col = X.iloc[:, X.columns.str.startswith(col)]
    
    for feat, func in SUMMARY_STATS.items():
        univariate_features[F'{col}_{feat}'] = X_col.apply(func, axis = 1)
        
univariate_features_df = pd.concat(univariate_features, axis=1)

In [14]:
univariate_features_df.head()

,PeakP_mean,PeakP_sdev,PeakD_mean,PeakD_sdev,Upcross_mean,Upcross_sdev,SWH_mean,SWH_sdev,SeaTemp_mean,SeaTemp_sdev,Hmax_mean,Hmax_sdev,THmax_mean,THmax_sdev,MCurDir_mean,MCurDir_sdev,MCurSpd_mean,MCurSpd_sdev
time,,,,,,,,,,,,,,,,,,
2022-01-02 00:00:00+00:00,7.926957,2.436289,215.147635,22.426363,4.881522,0.561486,187.717391,45.045346,10.075145,0.039775,298.413043,77.564511,6.998478,1.447617,144.831979,86.779029,0.134877,0.068940
2022-01-02 01:00:00+00:00,8.249783,2.387316,218.100335,19.539648,4.900652,0.534211,187.021739,45.801433,10.069312,0.040999,295.000000,80.520454,7.149348,1.376687,138.831023,89.017735,0.134565,0.068946
2022-01-02 02:00:00+00:00,8.583478,2.323690,220.965122,15.403543,4.918478,0.507531,186.760870,46.070567,10.062572,0.042364,295.152174,80.355249,7.305435,1.213707,133.363274,90.182891,0.136500,0.069302
2022-01-02 03:00:00+00:00,8.782391,2.116691,223.866221,7.731185,4.931739,0.482961,186.326087,46.526432,10.055978,0.044900,292.239130,82.843340,7.424783,1.060735,122.913839,84.519668,0.135768,0.068941
2022-01-02 04:00:00+00:00,9.019348,2.095215,225.570951,3.620865,4.935652,0.476323,185.586957,47.172591,10.047862,0.045245,290.826087,83.654388,7.345435,1.070239,110.675585,71.689996,0.135645,0.068785


#### Bivariate Feature Extraction
- Rolling binary statistics : Compute statistics that take pairs of variables as input. For example, the rolling covariance or rolling correlation  
- Rolling binary transformation followed by univariate statistics : Transform a pair of variables into a single variable, and summarise this variable. For example, computing the elementwise cross-correlation and then taking its average value.

In [15]:
from scipy import stats
import statsmodels.tsa.stattools as ts


def covariance(x: np.ndarray, y: np.ndarray) -> float:
    """ Covariance between x and y
    """
    cov_xy = np.cov(x, y)[0][1]

    return cov_xy


def co_integration(x: np.ndarray, y: np.ndarray):
    """ Co-integration test between x and y
    """
    r, _, _ = ts.coint(x, y)

    return r

In [16]:
import itertools

from scipy.spatial.distance import jensenshannon
from scipy import signal
from scipy.special import rel_entr

In [17]:
BIVARIATE_STATS = {
    'covariance': covariance,
    'co_integration': co_integration,
    'js_div': jensenshannon,
}

BIVARIATE_TRANSFORMATIONS = {
    'corr': signal.correlate,
    'conv': signal.convolve,
    'rel_entr': rel_entr,
}

In [18]:
# get all pairs of variables
col_combs = list(itertools.combinations(col_names, 2))
col_combs

[('PeakP', 'PeakD'),
 ('PeakP', 'Upcross'),
 ('PeakP', 'SWH'),
 ('PeakP', 'SeaTemp'),
 ('PeakP', 'Hmax'),
 ('PeakP', 'THmax'),
 ('PeakP', 'MCurDir'),
 ('PeakP', 'MCurSpd'),
 ('PeakD', 'Upcross'),
 ('PeakD', 'SWH'),
 ('PeakD', 'SeaTemp'),
 ('PeakD', 'Hmax'),
 ('PeakD', 'THmax'),
 ('PeakD', 'MCurDir'),
 ('PeakD', 'MCurSpd'),
 ('Upcross', 'SWH'),
 ('Upcross', 'SeaTemp'),
 ('Upcross', 'Hmax'),
 ('Upcross', 'THmax'),
 ('Upcross', 'MCurDir'),
 ('Upcross', 'MCurSpd'),
 ('SWH', 'SeaTemp'),
 ('SWH', 'Hmax'),
 ('SWH', 'THmax'),
 ('SWH', 'MCurDir'),
 ('SWH', 'MCurSpd'),
 ('SeaTemp', 'Hmax'),
 ('SeaTemp', 'THmax'),
 ('SeaTemp', 'MCurDir'),
 ('SeaTemp', 'MCurSpd'),
 ('Hmax', 'THmax'),
 ('Hmax', 'MCurDir'),
 ('Hmax', 'MCurSpd'),
 ('THmax', 'MCurDir'),
 ('THmax', 'MCurSpd'),
 ('MCurDir', 'MCurSpd')]

In [19]:
X.head()

,PeakP(t-23),PeakP(t-22),PeakP(t-21),PeakP(t-20),PeakP(t-19),PeakP(t-18),PeakP(t-17),PeakP(t-16),PeakP(t-15),PeakP(t-14),PeakP(t-13),PeakP(t-12),PeakP(t-11),PeakP(t-10),PeakP(t-9),...,MCurSpd(t-15),MCurSpd(t-14),MCurSpd(t-13),MCurSpd(t-12),MCurSpd(t-11),MCurSpd(t-10),MCurSpd(t-9),MCurSpd(t-8),MCurSpd(t-7),MCurSpd(t-6),MCurSpd(t-5),MCurSpd(t-4),MCurSpd(t-3),MCurSpd(t-2),MCurSpd(t-1)
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-01-02 00:00:00+00:00,4.010,4.130,4.085,5.985,5.970,7.180,8.515,7.380,7.380,7.275,7.415,9.030,6.255,9.890,9.030,...,0.097000,0.200500,0.165000,0.227833,0.240833,0.161833,0.219333,0.185333,0.091000,0.055500,0.031667,0.027667,0.033833,0.050667,0.077000
2022-01-02 01:00:00+00:00,4.130,4.085,5.985,5.970,7.180,8.515,7.380,7.380,7.275,7.415,9.030,6.255,9.890,9.030,6.645,...,0.200500,0.165000,0.227833,0.240833,0.161833,0.219333,0.185333,0.091000,0.055500,0.031667,0.027667,0.033833,0.050667,0.077000,0.129833
2022-01-02 02:00:00+00:00,4.085,5.985,5.970,7.180,8.515,7.380,7.380,7.275,7.415,9.030,6.255,9.890,9.030,6.645,9.005,...,0.165000,0.227833,0.240833,0.161833,0.219333,0.185333,0.091000,0.055500,0.031667,0.027667,0.033833,0.050667,0.077000,0.129833,0.170500
2022-01-02 03:00:00+00:00,5.985,5.970,7.180,8.515,7.380,7.380,7.275,7.415,9.030,6.255,9.890,9.030,6.645,9.005,6.905,...,0.227833,0.240833,0.161833,0.219333,0.185333,0.091000,0.055500,0.031667,0.027667,0.033833,0.050667,0.077000,0.129833,0.170500,0.161833
2022-01-02 04:00:00+00:00,5.970,7.180,8.515,7.380,7.380,7.275,7.415,9.030,6.255,9.890,9.030,6.645,9.005,6.905,8.195,...,0.240833,0.161833,0.219333,0.185333,0.091000,0.055500,0.031667,0.027667,0.033833,0.050667,0.077000,0.129833,0.170500,0.161833,0.221500


In [20]:
bivariate_features = []
for i, _ in X.iterrows():
    feature_set_i = {}
    for col1, col2 in col_combs:
        
        # getting the i-th instance for each column
        x1 = X.loc[i, X.columns.str.startswith(col1)]
        x2 = X.loc[i, X.columns.str.startswith(col2)]
        
        # compute each summary stat
        for feat, func in BIVARIATE_STATS.items():
            feature_set_i[f'{col1}|{col2}_{feat}'] = func(x1, x2)

        # for each transformation
        for trans_f, t_func in BIVARIATE_TRANSFORMATIONS.items():

            # apply transformation
            xt = t_func(x1, x2)

            # compute summary stat
            for feat, s_func in SUMMARY_STATS.items():
                feature_set_i[f'{col1}|{col2}_{trans_f}_{feat}'] = s_func(xt)
                
    bivariate_features.append(feature_set_i)

In [21]:
bivariate_features_df = pd.DataFrame(bivariate_features, index=X.index)
bivariate_features_df.head()

,PeakP|PeakD_covariance,PeakP|PeakD_co_integration,PeakP|PeakD_js_div,PeakP|PeakD_corr_mean,PeakP|PeakD_corr_sdev,PeakP|PeakD_conv_mean,PeakP|PeakD_conv_sdev,PeakP|PeakD_rel_entr_mean,PeakP|PeakD_rel_entr_sdev,PeakP|Upcross_covariance,PeakP|Upcross_co_integration,PeakP|Upcross_js_div,PeakP|Upcross_corr_mean,PeakP|Upcross_corr_sdev,PeakP|Upcross_conv_mean,...,THmax|MCurSpd_corr_mean,THmax|MCurSpd_corr_sdev,THmax|MCurSpd_conv_mean,THmax|MCurSpd_conv_sdev,THmax|MCurSpd_rel_entr_mean,THmax|MCurSpd_rel_entr_sdev,MCurDir|MCurSpd_covariance,MCurDir|MCurSpd_co_integration,MCurDir|MCurSpd_js_div,MCurDir|MCurSpd_corr_mean,MCurDir|MCurSpd_corr_sdev,MCurDir|MCurSpd_conv_mean,MCurDir|MCurSpd_conv_sdev,MCurDir|MCurSpd_rel_entr_mean,MCurDir|MCurSpd_rel_entr_sdev
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-01-02 00:00:00+00:00,39.761084,-2.161199,0.084459,20048.699745,11788.387305,20048.699745,11788.387305,NaN,NaN,0.439103,-1.695854,0.100605,454.888400,272.039759,454.888400,...,11.096450,7.220757,11.096450,7.220757,NaN,NaN,-0.293764,-1.784715,0.300047,229.638612,129.989191,229.638612,129.989191,NaN,NaN
2022-01-02 01:00:00+00:00,31.708169,-2.599988,0.083242,21151.540080,12287.423816,21151.540080,12287.423816,NaN,NaN,0.168168,-1.882183,0.102329,475.269059,281.724113,475.269059,...,11.309474,7.086952,11.309474,7.086952,NaN,NaN,-0.274899,-2.298385,0.309900,219.615253,121.983330,219.615253,121.983330,NaN,NaN
2022-01-02 02:00:00+00:00,21.562923,-2.484991,0.082586,22296.166481,12750.301453,22296.166481,12750.301453,NaN,NaN,-0.112164,-2.498686,0.103591,496.291944,290.371356,496.291944,...,11.722544,6.883511,11.722544,6.883511,NaN,NaN,-0.386216,-2.718442,0.318656,213.999154,113.548588,213.999154,113.548588,NaN,NaN
2022-01-02 03:00:00+00:00,8.193811,-1.020354,0.078507,23112.371523,13066.173222,23112.371523,13066.173222,NaN,NaN,-0.335016,-3.088304,0.099420,509.162063,295.507966,509.162063,...,11.850173,6.643352,11.850173,6.643352,NaN,NaN,-0.795135,-2.706203,0.326777,196.174129,104.280055,196.174129,104.280055,NaN,NaN
2022-01-02 04:00:00+00:00,4.463340,-4.337280,0.078376,23916.711508,13414.775018,23916.711508,13414.775018,NaN,NaN,-0.537371,-4.415226,0.102825,523.314587,300.732563,523.314587,...,11.712894,6.422811,11.712894,6.422811,NaN,NaN,-1.921494,-2.167718,0.339498,176.481238,97.302465,176.481238,97.302465,NaN,NaN


In [22]:
# concatenating all features with lags
X_with_features = pd.concat([X, univariate_features_df, bivariate_features_df], axis=1)
X_with_features.shape

(4790, 549)

In [23]:
# train/test split
X_train, X_test, y_train, y_test = train_test_split(X_with_features, y, test_size=0.3, shuffle=False)
# fitting a lgbm model without feature engineering
model_w_fe = MultiOutputRegressor(LGBMRegressor())
model_w_fe.fit(X_train, y_train)

# getting forecasts for the test set
preds_w_fe = model_w_fe.predict(X_test)

# computing the MAPE error
mape(y_test, preds_w_fe)

0.2501678505761729

- 성능이 약간 개선되었다.

#### Feature selection

In [24]:
# getting the importance of each feature in each horizon
avg_imp = pd.DataFrame([x.feature_importances_
                        for x in model_w_fe.estimators_]).mean()
avg_imp.head()

0    3.750000
1    4.083333
2    4.000000
3    3.250000
4    5.166667
dtype: float64

In [25]:
# getting the top 100 features
n_top_features = 100

importance_scores = pd.Series(dict(zip(X_train.columns, avg_imp)))
top_features = importance_scores.sort_values(ascending=False)[:n_top_features]
top_features_nm = top_features.index
top_features_nm

Index(['SWH(t-1)', 'SeaTemp_sdev', 'SWH|Hmax_js_div',
       'Upcross|MCurDir_covariance', 'PeakD|SeaTemp_covariance',
       'PeakP|Upcross_covariance', 'Hmax(t-1)', 'PeakD|Upcross_covariance',
       'MCurDir(t-1)', 'MCurDir|MCurSpd_covariance', 'PeakD(t-1)',
       'Upcross|SeaTemp_covariance', 'Upcross|THmax_covariance',
       'SeaTemp|MCurSpd_covariance', 'PeakD|MCurSpd_covariance',
       'Upcross|MCurSpd_covariance', 'Upcross|THmax_js_div',
       'PeakD|THmax_js_div', 'PeakP|SeaTemp_covariance',
       'PeakD|SeaTemp_js_div', 'PeakD|MCurDir_covariance', 'Upcross(t-1)',
       'PeakP|THmax_js_div', 'THmax|MCurDir_covariance',
       'PeakP|MCurSpd_covariance', 'PeakD|Upcross_js_div', 'PeakD_sdev',
       'MCurDir|MCurSpd_corr_sdev', 'Hmax|MCurSpd_js_div',
       'MCurDir|MCurSpd_js_div', 'MCurDir_sdev', 'THmax|MCurDir_js_div',
       'Upcross|MCurSpd_corr_sdev', 'SeaTemp|MCurDir_co_integration',
       'THmax|MCurSpd_covariance', 'PeakP|SeaTemp_corr_mean',
       'PeakP|MCurSpd

In [26]:
# subsetting training and testing sets by those features
X_train_top = X_train[top_features_nm]
X_test_top = X_test[top_features_nm]

# re-fitting the lgbm model
model_top_features = MultiOutputRegressor(LGBMRegressor())
model_top_features.fit(X_train_top, y_train)

# getting forecasts for the test set
preds_top_feats = model_top_features.predict(X_test_top)

# computing MAE error
mape(y_test, preds_top_feats)
# 0.229

0.25457869154188034

In [27]:
top_features

SWH(t-1)                      102.250000
SeaTemp_sdev                   31.500000
SWH|Hmax_js_div                29.250000
Upcross|MCurDir_covariance     28.833333
PeakD|SeaTemp_covariance       27.250000
                                 ...    
MCurDir(t-4)                    8.916667
PeakP|THmax_covariance          8.916667
SWH|SeaTemp_co_integration      8.916667
Upcross(t-6)                    8.916667
PeakD|MCurSpd_js_div            8.833333
Length: 100, dtype: float64

가장 중요한 특징은 목표 변수의 첫 번째 지연입니다. 그러나 추출된 특징 중 일부는 이 상위 15개에 포함되어 있습니다. 예를 들어, SWH|Hmax_js_div에서 세 번째로 좋은 특징이 있습니다. 이는 대상 변수의 지연과 Hmax의 지연 사이의 젠슨-섀넌 발산을 나타냅니다. 다섯 번째로 좋은 기능은 표준 편차 해수 온도 지연인 SeaTemp_sdev입니다. 이 공분산은 다양한 변수 쌍에 대한 관련 통계이기도 합니다.

중복된 특징을 제거하는 또 다른 방법은 상관관계 필터를 적용하는 것입니다. 상관관계가 높은 특징을 제거하여 데이터의 차원을 줄입니다.